# Import libraries

In [ ]:
#libraries import
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
try:
  from unidecode import unidecode
except:
  !pip install unidecode
  from unidecode import unidecode
from scipy.stats import norm
from datetime import datetime

from openpyxl.utils.cell import column_index_from_string, get_column_letter
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle
import openpyxl

import shutil

from google.colab import files

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Fazer upload de um arquivo
uploaded = files.upload()

!mkdir -p Mews_Partners

# Mover o arquivo para um diretório específico
shutil.move(list(uploaded.keys())[0], '/content/Mews_Partners/')

## Functions

In [ ]:
def fix_column_names(df):
    dict_fix = {'.':' ','-':' ','/':' ',' ':'_'}
    column_names = df.columns
    new_column_names = []
    for col in column_names:
        # Remove accents and special characters
        fixed_col = unidecode(col)
        for char in dict_fix:
            fixed_col = fixed_col.replace(char, dict_fix[char])
        new_column_names.append(fixed_col)
    return new_column_names

def classeABCQuantity(df, qty_col_name, price_col_name):
    # sum of quantities sold in each year
    df = df[df[qty_col_name]!=0].reset_index(drop=True)
    df_total_qty = df.groupby(['Code', 'Year'])[qty_col_name].sum().reset_index()
    df_total_price = df.groupby(['Code', 'Year'])[price_col_name].sum().reset_index()

    # cumulative sum for each year
    df_total_qty = df_total_qty.sort_values(['Year',qty_col_name], ascending=[True, False])
    df_total_qty['Cumulative_Sum'] = df_total_qty.groupby('Year')[qty_col_name].cumsum()
    df_total_qty['Cumulative_Percentage'] = df_total_qty['Cumulative_Sum'] / df_total_qty.groupby('Year')[qty_col_name].transform('sum')

    # classify the product in classes A,B and C
    df = pd.merge(df, df_total_qty[['Code', 'Year', 'Cumulative_Percentage']], on=['Code', 'Year'])
    abc_categories = pd.cut(df['Cumulative_Percentage'], bins=[0, 0.8, 0.95, 1.0], labels=['A', 'B', 'C'])
    df['Classe_ABC_quantity'] = abc_categories

    # cumulative sum for each year
    df_total_price = df_total_price.sort_values(['Year',price_col_name], ascending=[True, False])
    df_total_price['Cumulative_Sum_revenue'] = df_total_price.groupby('Year')[price_col_name].cumsum()
    df_total_price['Cumulative_Percentage_revenue'] = df_total_price['Cumulative_Sum_revenue'] / df_total_price.groupby('Year')[price_col_name].transform('sum')

    # classify the product in classes A,B and C
    df = pd.merge(df, df_total_price[['Code', 'Year', 'Cumulative_Percentage_revenue']], on=['Code', 'Year'])
    abc_categories = pd.cut(df['Cumulative_Percentage_revenue'], bins=[0, 0.8, 0.95, 1.0], labels=['A', 'B', 'C'])
    df['Classe_ABC_revenue'] = abc_categories

    return df

def fix_dates(df):
  # change date format
  df['Date'] = df['Year'].astype(str) + '-' + df['Month'].astype(str)
  df.Date = pd.to_datetime(df.Date, format='%Y-%m')
  return df$

def calculate_delay(df, list_col):
  # calculating the delay time
  dict_time = {'jrs': 1/30, 'semaine': 7/30, 'semaines': 7/30, 'mois':1}
  for col in list_col:
    df[col] = df[col].map(lambda x: x.split())
    df[col] = df[col].map(lambda x: int(x[0])*dict_time[x[1]])
  return df

# Import data

In [ ]:
# importing data
df_base_prev_passe_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=2, sheet_name = 'Previsions_historique').iloc[1:, 3:]
df_base_historique_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=3, sheet_name = 'Ventes_historique').iloc[:, 2:]
df_flux_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=2, sheet_name = 'Baseflux').iloc[1:, 2:]
df_base_article_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=1, sheet_name = 'Basearticle')
df_stock_original = pd.read_excel('Mews_Partners/DataModeleComplet.xlsx', header=3, sheet_name = 'Stocks').iloc[:, 2:]

# Analysis

In [ ]:
# Copying the original DataFrame
df_base_prev_passe = df_base_prev_passe_original.copy()

# Renaming columns
new_cols = ['Code', 'Marche', 'Canal', 'Code_Magasin', 'Year', 'Month', 'Forecast_Magasin']
df_base_prev_passe.columns = new_cols

# Copying and fixing data types for the DataFrame
df_base_prev = df_base_prev_passe.copy()
df_base_prev.Code = df_base_prev.Code.astype(int)
df_base_prev.Year = df_base_prev.Year.astype(int)
df_base_prev.Month = df_base_prev.Month.astype(int)

# Replacing values in the DataFrame
df_base_prev = df_base_prev.replace("nan", "").replace("-", "")

# Copying and renaming columns for the historical DataFrame
df_base_historique = df_base_historique_original.copy()
df_base_historique.rename(columns={'Code article ': 'Code', 'Marché': 'Marche', 'Code point de stock': 'Code_Magasin',
                                   'Année': 'Year', 'Mois ': 'Month', 'Quantité': 'Historique_vente_Magasin'}, inplace=True)

# Replacing values in the DataFrame
df_base_historique = df_base_historique.replace("nan", "")
df_base_historique.Canal = df_base_historique.Canal.astype(str)
df_base_historique.Code = df_base_historique.Code.astype(int)
df_base_historique.Year = df_base_historique.Year.astype(int)
df_base_historique.Month = df_base_historique.Month.astype(int)

# Copying and renaming columns for the flux DataFrame
df_flux = df_flux_original.copy()
list_cols_names_flux = ['Code', 'Marche','Canal','Delay_Magasin','Frequency_Magasin','MOQ_Magasin','Code_Magasin','Delay_France','Frequency_France',
                        'MOQ_France','Code_France','Delay_Monde','Frequency_Monde','MOQ_Monde','Code_Monde', 'Delay_Usine','Frequency_Usine','MOQ_Usine','Code_Usine']
df_flux.columns = list_cols_names_flux
df_flux = df_flux.replace("-", "")

# Copying, fixing column names, and data types for the article DataFrame
df_base_article = df_base_article_original.iloc[:, 1:].copy()
df_base_article.columns = fix_column_names(df_base_article)
df_base_article = df_base_article.rename(columns={'Code_article': 'Code', 'Classe_': 'Classe_base'})

# Copying and renaming columns for the stock DataFrame
df_stock = df_stock_original.copy()
df_stock.rename(columns={'Article': 'Code', 'Site ': 'Entrepot', 'Année': 'Year', 'Mois': 'Month'}, inplace=True)

# Merging DataFrames and fixing dates
df_base_prev_ventes = df_base_prev[['Code', 'Marche', 'Year', 'Month', 'Forecast_Magasin']].merge(df_base_historique, on=['Code', 'Marche', 'Year', 'Month'])
df_base_prev_ventes = fix_dates(df_base_prev_ventes)
df_stock = fix_dates(df_stock).drop(['Year', 'Month'], axis=1)

# Merging DataFrames and fixing entrepôt names
list_marche = {'France': 'FR', 'Belgique': 'BE', 'Belgium': 'BE', 'Spain': 'ES', 'Turkey': 'TK', 'Germany': 'DE',
                  'Portugal': 'PT', 'Italy': 'IT', 'UK': 'UK', 'Swiss': 'CH', 'Poland': 'PL'}
list_entrepot = {'FR': 'Central', 'BE': 'Central', 'TK': 'Central'}

df_temp = df_base_prev_ventes.copy()
df_temp['Marche'] = df_temp['Marche'].replace(list_marche)
df_temp['Entrepot'] = df_temp['Marche'].replace(list_entrepot)


df_prev_ventes_stock = df_temp.merge(df_stock, on=['Code', 'Date','Entrepot'])

df_flux['Marche'] = df_flux['Marche'].replace(list_marche)

# Filtering out rows with Code 'Loading...'
df_flux = df_flux[df_flux.Code != "Loading..."]
df_prev_ventes_stock = df_prev_ventes_stock[df_prev_ventes_stock.Code != "Loading..."]

## Fix flux As-Is table

In [ ]:
# Create a list of markets where the Entrepot is 'Central'
list_not_local_entrepot = ['FR', 'BE','TK']

# Filter and select columns for the local DataFrame
df_flux_local = df_flux.loc[~(df_flux.Marche.isin(list_not_local_entrepot)), ['Code', 'Marche', 'Canal', 'Delay_Magasin', 'Frequency_Magasin', 'MOQ_Magasin', 'Code_Magasin']]

# Map the 'Entrepot' column based on the 'Marche' column values
df_flux_local.loc[:, 'Entrepot'] = df_flux_local['Marche'].map(lambda x: x if x not in list_not_local_entrepot else "Central")

# Reset index, drop duplicates, and rename columns for the local DataFrame
df_flux_local.reset_index(drop=True, inplace=True)
df_flux_local.drop_duplicates(inplace=True)
df_flux_local = df_flux_local.rename(columns={'Delay_Magasin':'Delay_Entrepot', 'Frequency_Magasin':'Frequency_Entrepot', 'MOQ_Magasin':'MOQ_Entrepot', 'Code_Magasin':'Code_Entrepot'})

# Filter and select columns for the central DataFrame
df_flux_central = df_flux.loc[df_flux['Code_Monde'] != '', ['Code', 'Marche', 'Canal', 'Delay_Monde', 'Frequency_Monde', 'MOQ_Monde', 'Code_Monde']]

# Drop duplicates, add 'Entrepot' column with value 'Central', reset index, and rename columns for the central DataFrame
df_flux_central.drop_duplicates(inplace=True)
df_flux_central['Entrepot'] = 'Central'
df_flux_central.reset_index(drop=True, inplace=True)
df_flux_central = df_flux_central.rename(columns={'Delay_Monde':'Delay_Entrepot', 'Frequency_Monde':'Frequency_Entrepot', 'MOQ_Monde':'MOQ_Entrepot', 'Code_Monde':'Code_Entrepot'})

# Concatenate the central and local DataFrames
df_flux_entrepot = pd.concat([df_flux_central, df_flux_local], axis=0, ignore_index=True)

# dropping duplicate lines
df_flux_entrepot = df_flux_entrepot[['Code','Canal', 'Delay_Entrepot', 'Frequency_Entrepot',
       'MOQ_Entrepot', 'Code_Entrepot', 'Entrepot']].drop_duplicates()

## Fill not available data in forecast with its mean

In [ ]:
# Creating a copy of the DataFrame
df_prev_ventes_stock_clean = df_prev_ventes_stock.copy()

# If 'Historique_vente_Magasin' is less than or equal to 0 and 'Forecast_Magasin' is NaN, set 'Forecast_Magasin' to 0
df_prev_ventes_stock_clean.loc[
    (df_prev_ventes_stock_clean['Historique_vente_Magasin'] <= 0) & (df_prev_ventes_stock_clean['Forecast_Magasin'].isna()),
    'Forecast_Magasin'] = 0

# Calculating the mean value of 'Forecast_Magasin' respecting the product, entrepot, and year
mean_forecast = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Forecast_Magasin'] > 0].groupby(
    ['Code', 'Entrepot', 'Year','Marche'])[['Forecast_Magasin']].mean().reset_index()

# Creating two DataFrames: one with positive 'Forecast_Magasin' values and one with NaN 'Forecast_Magasin' values
df_forecast_pos = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Forecast_Magasin'] > 0].copy()
df_forecast_nan = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Forecast_Magasin'].isna()].copy()

# Merging the DataFrame with NaN 'Forecast_Magasin' with the mean forecast values
df_forecast_nan_mean = df_forecast_nan.drop('Forecast_Magasin', axis=1).merge(mean_forecast,
                                                                             on=['Code', 'Entrepot', 'Year','Marche'])
# Concatenating the DataFrames with positive 'Forecast_Magasin' and mean forecast values for NaN 'Forecast_Magasin'
df_merge_forecast = pd.concat([df_forecast_pos, df_forecast_nan_mean]).drop('Historique_vente_Magasin', axis=1)


In [ ]:
# Calculating the mean value of 'Historique_vente_Magasin' respecting the product, entrepot, and year
mean_ventes = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Historique_vente_Magasin'] > 0].groupby(
    ['Code', 'Entrepot', 'Year','Marche'])[['Historique_vente_Magasin']].mean().reset_index()

# Creating two DataFrames: one with positive 'Historique_vente_Magasin' values and one with NaN 'Historique_vente_Magasin' values
df_ventes_pos = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Historique_vente_Magasin'] > 0].copy()
df_ventes_nan = df_prev_ventes_stock_clean[df_prev_ventes_stock_clean['Historique_vente_Magasin'].isna()].copy()

# Merging the DataFrame with NaN 'Historique_vente_Magasin' with the mean forecast values
df_ventes_nan_mean = df_ventes_nan.drop('Historique_vente_Magasin', axis=1).merge(mean_ventes,
                                                                             on=['Code', 'Entrepot', 'Year','Marche'])
# Concatenating the DataFrames with positive 'Historique_vente_Magasin' and mean ventes values for NaN 'Historique_vente_Magasin'
df_merge_ventes = pd.concat([df_ventes_pos, df_ventes_nan_mean]).drop('Forecast_Magasin', axis=1)

# merging both dataframes
df_merge_ventes_forecast = df_merge_ventes.merge(df_merge_forecast, on=['Code', 'Marche', 'Year', 'Month', 'Canal',
       'Code_Magasin', 'Date', 'Entrepot',
       'Stock'])

## Joining data in entrepôt central and local

In [ ]:
# Filling NaN values in the 'Code_Magasin' column with an empty string
df_merge_ventes_forecast.Code_Magasin = df_merge_ventes_forecast.Code_Magasin.fillna("")

# Grouping by multiple columns and aggregating the sum of 'Forecast_Magasin' and 'Historique_vente_Magasin'
df_forecast_local_central = df_merge_ventes_forecast.groupby([
    'Code', 'Year', 'Month', 'Entrepot', 'Canal', 'Code_Magasin', 'Date', 'Stock']).aggregate({
    'Forecast_Magasin': 'sum',
    'Historique_vente_Magasin': 'sum'}).reset_index()

# Dropping duplicates from the resulting DataFrame
df_forecast_local_central.drop_duplicates(inplace=True)

## Calculate the Standard deviation

In [ ]:
# Creating a copy of the DataFrame
df_merge_forecast_clean = df_forecast_local_central.copy()

# Merging with additional information from df_base_article
df_forecast_article = df_merge_forecast_clean.merge(df_base_article[['Code','Prix_de_vente','PRI','Classe_base']], on='Code')

# Calculating revenue for historical and forecasted sales
df_forecast_article['Revenue_historic'] = df_forecast_article['Historique_vente_Magasin'] * df_forecast_article['Prix_de_vente']
df_forecast_article['Revenue_forecasted'] = df_forecast_article['Forecast_Magasin'] * df_forecast_article['Prix_de_vente']

# Selecting only data with positive values for both 'Historique_vente_Magasin' and 'Forecast_Magasin'
df_diff = df_forecast_article.loc[(df_forecast_article.Historique_vente_Magasin > 0) & (df_forecast_article.Forecast_Magasin > 0)][[
    'Code', 'Historique_vente_Magasin', 'Entrepot', 'Forecast_Magasin', 'Revenue_historic', 'Revenue_forecasted', 'Year', 'Month']].copy()

# Calculating standard deviation for quantity and revenue differences
df_diff['Std_deviation_qty'] = df_diff['Forecast_Magasin'] - df_diff['Historique_vente_Magasin']
df_diff['Std_deviation_rev'] = df_diff['Revenue_forecasted'] - df_diff['Revenue_historic']

# Grouping by 'Code', 'Entrepot', and 'Year' and calculating standard deviation for quantity and revenue differences
df_diff_std = df_diff.groupby(['Code', 'Entrepot', 'Year'])[['Std_deviation_qty', 'Std_deviation_rev']].std().reset_index()

# Merging DataFrames
df_base_std = pd.merge(df_forecast_article, df_diff_std, on=['Entrepot', 'Code', 'Year'])

# Filtering out rows with NaN values in 'Std_deviation_qty' and 'Std_deviation_rev'
df_base_std = df_base_std[(~df_base_std.Std_deviation_qty.isna()) & (~df_base_std.Std_deviation_rev.isna())]

# adding the info of Marche for each code in df_base_std
df_marche = df_merge_ventes_forecast[['Code','Marche','Year','Entrepot']].drop_duplicates()
df_base_std = df_base_std.merge(df_marche, on=['Code','Year','Entrepot'])

## Calculate the Stock Roulant

In [ ]:
# Merge relevant columns from df_base_std and df_flux_entrepot DataFrames
df_std_flux = df_base_std[['Code', 'Year', 'Month', 'Forecast_Magasin','Historique_vente_Magasin','Revenue_historic','Revenue_forecasted',
                           'Date', 'Entrepot','Stock', 'Std_deviation_qty','Std_deviation_rev','Prix_de_vente', 'PRI', 'Classe_base']].merge(
                               df_flux_entrepot, on=['Code','Entrepot'])

# List of columns to be filled with default values and then converted to numeric
list_cols = ['Delay_Entrepot','Frequency_Entrepot']

# Replace empty strings with "0 jrs" and fill NaN values with "0 jrs"
df_std_flux[list_cols] = df_std_flux[list_cols].replace("", "0 jrs").fillna("0 jrs")

# Calculate delay based on specific columns
df_std_delay = calculate_delay(df_std_flux, list_cols)

# Calculate the mean forecast for each product, year, and entrepot
df_mean_ventes = df_std_delay[df_std_delay.Historique_vente_Magasin > 0].groupby(['Code','Year','Entrepot'])[[
    'Historique_vente_Magasin']].mean().reset_index().rename(columns={'Historique_vente_Magasin':'Mean_ventes'})

# Merge the mean forecast back to the DataFrame
df_stock_roulant = df_std_delay.merge(df_mean_ventes, on=['Code','Year','Entrepot'])

# Calculate the stock roulant by multiplying the mean forecast in a year by frequency divided by 2
df_stock_roulant['Stock_roulant'] = df_stock_roulant['Frequency_Entrepot']*df_stock_roulant['Mean_ventes']/2

## Calculate the ABC class for each product

In [ ]:
# Call a function to classify products based on quantity and revenue
df_classes = classeABCQuantity(df_stock_roulant, 'Historique_vente_Magasin', 'Revenue_historic')[
    ['Code', 'Year','Cumulative_Percentage', 'Classe_ABC_quantity', 'Cumulative_Percentage_revenue',
       'Classe_ABC_revenue']].drop_duplicates(subset=['Code','Year'])

# Merge the classification information back to the DataFrame
df_merge_forecast_std = df_stock_roulant.merge(df_classes, on=['Code','Year'])

## Update Google Sheets file

In [ ]:
# df to save in google sheets
df_save = df_merge_forecast_std[['Date', 'Entrepot', 'Code',
       'Historique_vente_Magasin', 'Forecast_Magasin', 'Stock',
       'Std_deviation_rev',
       'Classe_ABC_quantity',
       'Classe_ABC_revenue', 'Canal', 'Delay_Entrepot',
       'Frequency_Entrepot', 'Stock_roulant','Std_deviation_qty','Revenue_historic','Revenue_forecasted','Mean_ventes']]

df_save.drop_duplicates(subset=['Date', 'Entrepot', 'Code'],inplace=True)

# name of the google sheets
spreadsheet = gc.open('Pole_Mews_Final')
worksheet = spreadsheet.worksheet('Stock Roulant')

df_save['Date'] = df_save['Date'].astype(str)
df_save = df_save.fillna('')
worksheet.update('A1', [df_save.columns.values.tolist()] + df_save.values.tolist())